In [31]:
import pandas as pd
import re
from collections import Counter
from langchain.llms import Ollama
from langchain.schema import SystemMessage, HumanMessage
from tqdm import tqdm
import time

# === Load Data ===
het_df = pd.read_csv('qualitative_heterogeneous_full.csv')
homo_df = pd.read_csv('qualitative_homogeneous_full.csv')
het_df['setup'] = 'heterogeneous'
homo_df['setup'] = 'homogeneous'
df = pd.concat([het_df, homo_df], ignore_index=True)

# === Constants ===
VALID_CODES = {"JUST", "LEG", "ACC", "TRAN", "CONS", "HARM", "RGHT", "UTIL", "RESP", "SOLI"}
MODELS = ["llama3", "gemma3"]

# === Define System Prompt ===
system_prompt = SystemMessage(content="""
SYSTEM:
You are a legal reasoning analyst. Your job is to read a short justification and assign **exactly one** of the following jurisprudential themes, based on the text’s primary concern. Return **only** the theme code—nothing else.

Themes (code – name – key markers):
- JUST – Fairness / Justice  
  Equity, procedural fairness, non-discrimination, equal treatment  
- LEG – Legality / Rule of Law  
  Formal compliance, legal validity, statutory norms, precedent  
- ACC – Accountability  
  Assignable blame, institutional oversight, who is held responsible  
- TRAN – Transparency  
  Openness, explainability, public access to rationale or data  
- CONS – Consent / Autonomy  
  Voluntary agreement, informed choice, personal control, opt-in rights  
- HARM – Harm / Risk  
  Preventing or mitigating physical, financial, or social harm  
- RGHT – Rights-based Reasoning  
  Liberty, dignity, privacy, constitutional or human rights  
- UTIL – Utility / Welfare  
  Maximizing aggregate benefit, efficiency, cost-benefit analysis  
- RESP – Responsibility / Liability  
  Legal or moral liability, duty of care, answerability for outcomes  
- SOLI – Solidarity / Common Good  
  Collective ethics, public interest, environmental or societal wellbeing

**Few-Shot Examples**

1. Text:
   “This rule ensures that all parties are treated equitably and have fair access to the process.”
   → JUST

2. Text:
   “The proposed regulation fully complies with existing statutes and established case law.”
   → LEG

3. Text:
   “Our audit framework holds managers accountable for any deviations from policy.”
   → ACC

4. Text:
   “The algorithm’s decision-making steps must be transparent and explainable to stakeholders.”
   → TRAN

5. Text:
   “Users must explicitly opt in and provide informed consent before any data collection.”
   → CONS

6. Text:
   “We need to limit exposure to hazardous chemicals to reduce the risk of injury.”
   → HARM

7. Text:
   “This policy protects individuals’ right to privacy by safeguarding personal data.”
   → RGHT

8. Text:
   “Adopting this approach maximizes overall social welfare while minimizing costs.”
   → UTIL

9. Text:
   “The manufacturer is legally liable for any defects under product liability law.”
   → RESP

10. Text:
    “This initiative promotes environmental sustainability for the benefit of future generations.”
    → SOLI

---

When you receive new input, you will be shown:

HUMAN:


Respond only with the theme code (e.g., JUST, ACC, HARM). Do not explain or return multiple codes.
""")

def format_prompt(text_type, content):
    return HumanMessage(content=f"""
Classify the dominant theme of the following {text_type}.

Text:
\"\"\"{content.strip()}\"\"\"

Respond with the single theme code.
""")

def extract_code(response_text):
    cleaned = re.sub(r"[^A-Za-z]", " ", response_text).strip().upper()
    tok = cleaned.split()[0] if cleaned else ""
    return tok if tok in VALID_CODES else "UNKNOWN"

# === Export per-model files ===
for model in MODELS:
    llm = Ollama(model=model)
    rows = []
    for _, row in tqdm(df.iterrows(), total=len(df), desc=f"Processing {model}"):
        rec = row.to_dict()

        # Define which columns to use, with fallbacks
        tasks = [
            ("rule",       "parsed_rule",              None,                       "proposed rule"),
            ("reasoning",   "parsed_reasoning",         "raw_proposal_reasoning",   "proposal reasoning"),
            ("voting",     "parsed_voting_reasoning",  "raw_voting_reasoning",     "voting justification"),
        ]

        for aspect, parsed_col, raw_col, text_type in tasks:
            # pick parsed if non-empty, else raw (if provided)
            text = rec.get(parsed_col, "")
            if (not isinstance(text, str)) or len(text.strip()) < 5:
                if raw_col:
                    text = rec.get(raw_col, "")
                else:
                    text = ""

            key = f"theme_{aspect}_{model}"
            if not isinstance(text, str) or len(text.strip()) < 5:
                rec[key] = "EMPTY"
            else:
                messages = [system_prompt, format_prompt(text_type, text)]
                try:
                    resp = llm.invoke(messages)
                    rec[key] = extract_code(str(resp))
                except Exception:
                    rec[key] = "ERROR"
                time.sleep(0.2)

        rows.append(rec)
    pd.DataFrame(rows).to_csv(f"themes_{model}.csv", index=False)



Processing llama3:   0%|                               | 0/2200 [00:04<?, ?it/s]


KeyboardInterrupt: 

In [32]:
import pandas as pd
from collections import Counter

# Load both theme files
df_llama = pd.read_csv('./themes_llama3.csv')
df_gemma = pd.read_csv('./themes_gemma3.csv')

# Determine common key columns (all non-theme columns)
all_cols = set(df_llama.columns) & set(df_gemma.columns)
key_cols = [col for col in all_cols if not col.startswith('theme_')]

# Merge on identifiers to align rows
merged = pd.merge(df_llama, df_gemma, on=key_cols)

# Manual Cohen's kappa calculation
def cohen_kappa(labels1, labels2):
    n = len(labels1)
    # Unique labels
    labels = sorted(set(labels1) | set(labels2))
    # Confusion counts
    counts = {l: {k: 0 for k in labels} for l in labels}
    for a, b in zip(labels1, labels2):
        counts[a][b] += 1
    # Observed agreement
    po = sum(counts[l][l] for l in labels) / n
    # Marginal probabilities
    row_totals = {l: sum(counts[l].values()) for l in labels}
    col_totals = {l: sum(counts[r][l] for r in labels) for l in labels}
    pe = sum((row_totals[l]/n) * (col_totals[l]/n) for l in labels)
    return (po - pe) / (1 - pe) if pe != 1 else 1.0

# Compute kappa for each aspect
kappa_results = {}
for aspect in ['rule', 'reasoning', 'voting']:
    col1 = f'theme_{aspect}_llama3'
    col2 = f'theme_{aspect}_gemma3'
    kappa_results[aspect] = cohen_kappa(merged[col1], merged[col2])

# Display results
kappa_df = pd.DataFrame.from_dict(kappa_results, orient='index', columns=["Cohen's kappa"])
kappa_df.index.name = 'Aspect'

print(kappa_df)


           Cohen's kappa
Aspect                  
rule            0.553490
reasoning       0.417151
voting          0.638050


In [29]:
import pandas as pd

# 1. Load both model outputs (they must be in the same row order)
llama3 = pd.read_csv('themes_llama3.csv')
gemma3 = pd.read_csv('themes_gemma3.csv')

# 2. (Optional) Verify they’re aligned by comparing some columns
assert (llama3['vignette_id'] == gemma3['vignette_id']).all()
assert (llama3['agent_id']    == gemma3['agent_id']).all()

# 3. Sample 10% of the rows by position
frac = 0.10
sample = llama3.sample(frac=frac, random_state=42).copy()
# Use the same index selection on gemma3
sample_idx = sample.index
sample_gemma = gemma3.loc[sample_idx].copy()

# 4. Prepare a human‐annotation file
#    Grab any text fields you want annotated (e.g. parsed_rule, parsed_reasoning, parsed_voting_reasoning)
texts = llama3.loc[sample_idx, [
    'parsed_rule', 'parsed_reasoning', 'parsed_voting_reasoning','raw_voting_reasoning','raw_proposal_reasoning'
]].rename(columns={
    'parsed_rule': 'text_rule',
    'parsed_reasoning': 'text_reasoning',
    'parsed_voting_reasoning': 'text_voting'
})

# 5. Build one DataFrame with both models’ pre-annotations plus blank gold columns
gold = pd.DataFrame({
    'text_rule': texts['text_rule'],
    'text_reasoning': texts['text_reasoning'],
    'text_voting': texts['text_voting'],
    'raw_proposal_reasoning': texts['raw_proposal_reasoning'],
    'raw_voting_reasoning': texts['raw_voting_reasoning'],
    'pred_rule_llama3': sample['theme_rule_llama3'],
    'pred_reasoning_llama3': sample['theme_reasoning_llama3'],
    'pred_voting_llama3': sample['theme_voting_llama3'],
    'pred_rule_gemma3': sample_gemma['theme_rule_gemma3'],
    'pred_reasoning_gemma3': sample_gemma['theme_reasoning_gemma3'],
    'pred_voting_gemma3': sample_gemma['theme_voting_gemma3'],
    # Empty columns for your human labels:
    'gold_rule': '',
    'gold_reasoning': '',
    'gold_voting': ''
}, index=sample_idx)

# 6. Export for annotation
gold.to_csv('gold_sample_both_models.csv', index=False)
#print("Wrote gold_sample_both_models.csv with", len(gold), "rows. Please fill gold_* columns by hand.")


In [35]:
# compare human baseline with llama3 and gemma3

import pandas as pd
from sklearn.metrics import cohen_kappa_score

# Load the combined file with model predictions and human annotations
df = pd.read_csv('./gold_sample_both_models.csv') # after the human has annotated the file
print("length", len(df))

# Compute Cohen's kappa for human vs each model
aspects = ['rule', 'reasoning', 'voting']
results = []

for aspect in aspects:
    human_col = f'gold_{aspect}'
    llama_col = f'pred_{aspect}_llama3'
    gemma_col = f'pred_{aspect}_gemma3'
    
    kappa_llama = cohen_kappa_score(df[human_col], df[llama_col])
    kappa_gemma = cohen_kappa_score(df[human_col], df[gemma_col])
    
    results.append({
        'Aspect': aspect,
        'Human vs Llama3': kappa_llama,
        'Human vs Gemma3': kappa_gemma
    })

# Create results DataFrame
kappa_df = pd.DataFrame(results).set_index('Aspect')

print(kappa_df)


length 220
           Human vs Llama3  Human vs Gemma3
Aspect                                     
rule              0.759838         0.715221
reasoning         0.706314         0.613386
voting            0.839059         0.815939


In [ ]:
# As kappa values for llama3 looks better, we will use Llama3 thematic groupings.